# 常见文档（txt, pdf, csv等）的加载与分割

详细文档见[rag_example](./rag_example/)

这里如果是windows可能会报错：ModuleNotFoundError: No module named 'pwd'

解决：

```bash
pip install langchain==0.1.6 langchain-community==0.0.19 langchain-core==0.1.23
```

## 文档的加载

In [6]:
from langchain.document_loaders import CSVLoader

loader = CSVLoader('rag_example/about.csv')
loader.load()


[Document(page_content='序号: 1\n问题: 《双城之战》第二季什么时候播出？\n答案: 第二季于2024年11月上线，大结局于11月23日播出。', metadata={'source': 'rag_example/about.csv', 'row': 0}),
 Document(page_content='序号: 2\n问题: 《双城之战》第二季共有多少集？\n答案: 第二季共有9集。', metadata={'source': 'rag_example/about.csv', 'row': 1}),
 Document(page_content='序号: 3\n问题: 《双城之战》的故事背景是什么？\n答案: 故事背景设定在皮尔特沃夫与祖安，围绕科技与信念冲突展开。', metadata={'source': 'rag_example/about.csv', 'row': 2}),
 Document(page_content='序号: 4\n问题: 《双城之战》的主要角色有哪些？\n答案: 蔚（Vi）、金克丝（Jinx）、杰斯（Jayce）、凯特琳（Caitlyn）等。', metadata={'source': 'rag_example/about.csv', 'row': 3}),
 Document(page_content='序号: 5\n问题: 《双城之战》是哪个工作室制作的？\n答案: 由法国动画工作室Fortiche制作，拳头游戏监督。', metadata={'source': 'rag_example/about.csv', 'row': 4}),
 Document(page_content='序号: 6\n问题: 《双城之战》是否会有第三季？\n答案: 第二季为最终季，目前没有第三季计划。', metadata={'source': 'rag_example/about.csv', 'row': 5}),
 Document(page_content='序号: 7\n问题: 《双城之战》的视觉效果如何？\n答案: 第二季视觉效果获高度评价，动画制作极为精美。', metadata={'source': 'rag_example/about.csv', 'row': 6}),
 Document(page_con

可见，csv的读取就是按行读取，并且在每行拼接了表名

In [8]:
# pip install pymupdf
from langchain.document_loaders import TextLoader, PyMuPDFLoader

loader1 = PyMuPDFLoader('rag_example/about.pdf')
loader2 = TextLoader('rag_example/about.txt')
loader1.load(), loader2.load()

([Document(page_content='英雄联盟：双城之战》第二季简介\n《英雄联盟：双城之战》（Arcane）是拳头公司（Riot Games）与法国动画工作室Fortiche联合制作的\n原创动画剧集，基于英雄联盟的背景世界打造。第二季于2024年11月上线，共包含9集，继续讲述了皮\n尔特沃夫（Piltover）与祖安（Zaun）之间复杂的矛盾与冲突。\n剧情概述\n第二季聚焦蔚（Vi）与金克丝（Jinx）之间复杂的情感与对立关系，同时引入了更多英雄联盟宇宙的重\n要角色。剧中呈现了皮尔特沃夫和祖安两个城市在科技与魔法交织下的社会矛盾，以及在不断扩展的英\n雄联盟宇宙中关于权力与自由的讨论。\n角色与制作\n剧集主要角色包括：\n蔚（Vi）： 执着且勇敢的地下拳手。\n金克丝（Jinx）： 具有不稳定精神状态的疯狂发明家。\n杰斯（Jayce）： 希望用科技改变世界的理想主义者。\n凯特琳（Caitlyn）： 皮尔特沃夫的警探，与蔚有紧密合作关系。\n动画由Fortiche高水准制作，视觉效果震撼，细节处理到位，获得了观众的一致好评。\n第二季亮点\n视觉体验： 动画细节与风格再次升级，打造了充满未来感与蒸汽朋克风格的符文之地。\n情感张力： 角色间的互动和冲突极具感染力，情节紧张且深刻。\n伏笔与彩蛋： 每一集都埋藏了与英雄联盟世界观相关的细节，吸引了大批游戏粉丝。\n未来计划\n拳头公司确认第二季为《双城之战》的最终季，目前没有推出第三季的计划。然而，英雄联盟宇宙的影\n视化项目仍在持续推进，后续剧集将探索符文之地其他地区的故事，包括艾欧尼亚、诺克萨斯和德玛西\n亚等。\n观众评价\n第二季播出后在全球范围内引发热烈讨论，其精美的动画制作、紧凑的剧情和对角色内心的刻画收获了\n观众的高度评价。不过，大结局也引发了一些争议，部分观众对剧情发展表达了不同意见。\n', metadata={'source': 'rag_example/about.pdf', 'file_path': 'rag_example/about.pdf', 'page': 0, 'total_pages': 2, 'format': 'PDF 1.4', 'title': 'a', 'author': '', 'subject': '', 'keywords': '', 'c

## 文档分割

加载完成后，需要对文档进行分割，大段文字分割为小段是有利于后续做索引的。分割时，需要指定分割的 chunk 长度，和重叠部分的长度，适当的 chunk 大小和重叠，可以提升处理长文本的流畅性和连贯性。如果没有重叠部分，相当于链表的链断了，各个文本块之间是孤立无联系的。

In [10]:
from langchain.text_splitter import RecursiveCharacterTextSplitter

text_splitter = RecursiveCharacterTextSplitter(
    # separator = ['\n\n', '\n', ' ', ''],  # 默认分隔符为换行和空格； ‘’ 表示按字切，即无指定分隔符
    chunk_size = 50,  # 文本块大小，default:4000，一般为500
    chunk_overlap = 20,  # 文本块重叠部分大小， default:200， 一般为100 or 200
    )

loader = TextLoader('rag_example/about.txt')
documents = loader.load_and_split(text_splitter)

documents,len(documents)

([Document(page_content='## 英雄联盟：双城之战》第二季简介', metadata={'source': 'rag_example/about.txt'}),
  Document(page_content='《英雄联盟：双城之战》（Arcane）是拳头公司（Riot', metadata={'source': 'rag_example/about.txt'}),
  Document(page_content='Games）与法国动画工作室Fortiche联合制作的原创动画剧集，基于英雄联盟的背景世界打造。第', metadata={'source': 'rag_example/about.txt'}),
  Document(page_content='动画剧集，基于英雄联盟的背景世界打造。第二季于2024年11月上线，共包含9集，继续讲述了皮尔特沃夫', metadata={'source': 'rag_example/about.txt'}),
  Document(page_content='月上线，共包含9集，继续讲述了皮尔特沃夫（Piltover）与祖安（Zaun）之间复杂的矛盾与冲突。', metadata={'source': 'rag_example/about.txt'}),
  Document(page_content='### 剧情概述', metadata={'source': 'rag_example/about.txt'}),
  Document(page_content='第二季聚焦蔚（Vi）与金克丝（Jinx）之间复杂的情感与对立关系，同时引入了更多英雄联盟宇宙的重要', metadata={'source': 'rag_example/about.txt'}),
  Document(page_content='立关系，同时引入了更多英雄联盟宇宙的重要角色。剧中呈现了皮尔特沃夫和祖安两个城市在科技与魔法交织下的', metadata={'source': 'rag_example/about.txt'}),
  Document(page_content='特沃夫和祖安两个城市在科技与魔法交织下的社会矛盾，以及在不断扩展的英雄联盟宇宙中关于权力与自由的讨论', metadata={'source':